**Parking Data**

In [1]:
from math import radians, cos, sin, asin, sqrt
import json
import glob

combined = []
for json_file in glob.glob("Parking/*/*/*/*.json"): #Assuming that your json files and .py file in the same directory
    with open(json_file, "rb") as infile:
        combined.append(json.load(infile))

In [2]:
import pandas as pd
#make json file into pd dataframe
df = pd.json_normalize(combined, record_path =['features'])
# drop irrelevant columns
df_selected = df.drop(['geometry.type', 'properties.State', 'properties.Type', 'type'], axis= 1)
df_selected

,Id,geometry.coordinates,properties.Name,properties.PubDate,properties.FreeSpaceShort,properties.FreeSpaceLong,properties.ShortCapacity,properties.LongCapacity
0,02900c40-0e95-4369-b9c4-22b2233a3440,"[4.915205, 52.377864]",CE-P28 PTA Touringcars,2021-01-03T17:27:42.028Z,42,,42,
1,900000118_parkinglocation,"[4.92911, 52.35694]",CE-P20 Oostpoort,2021-01-03T17:27:42.028Z,350,0,373,0
2,900000001_parkinglocation,"[4.8536, 52.343]",CE-P02 P Olympisch stadion,2021-01-03T17:27:42.028Z,305,249,309,290
3,900000117_parkinglocation,"[4.96043, 52.37135]",CE-P16 P+R Zeeburg 1,2021-01-03T17:27:42.028Z,61,0,210,0
4,901000105_parkinglocation,"[4.864, 52.3338]",ZD-P3 VU campus,2021-01-03T17:27:42.028Z,529,,543,
...,...,...,...,...,...,...,...,...
5985056,900000013_parkinglocation,"[4.86627, 52.38462]",CE-P14 Westerpark,2020-12-25T11:21:41.980Z,38,103,267,180
5985057,900000109_parkinglocation,"[4.84537, 52.37933]",CE-P18 P+R Bos en Lommer,2020-12-25T11:21:41.980Z,114,0,214,0
5985058,900000007_parkinglocation,"[4.89459, 52.37639]",CE-P08 De Kolk,2020-12-25T11:21:41.980Z,189,6,300,75
5985059,900000082_parkinglocation,"[4.94441, 52.3125]",ZO-P06 Pathe/HMH,2020-12-25T11:21:41.980Z,388,5,389,5


In [13]:
df_selected= df_selected.drop_duplicates(subset=['properties.Name', 'properties.PubDate'])
df_selected_sorted = df_selected.sort_values(by='properties.PubDate')
df_selected_sorted['properties.PubDate'] = pd.to_datetime(df_selected_sorted['properties.PubDate'], utc=True)
Parking_coordinates = df_selected_sorted.copy()


In [14]:
df_selected_sorted.set_index('properties.PubDate', inplace=True)
cols_float = df_selected_sorted.columns.to_list()[3:]
df_selected_sorted[cols_float] = df_selected_sorted[cols_float].apply(pd.to_numeric, errors='coerce', axis=1)
df_selected_sorted['capacity.filled'] = (df_selected_sorted['properties.ShortCapacity'] - df_selected_sorted['properties.FreeSpaceShort']) / df_selected_sorted['properties.ShortCapacity']

df_selected_sorted = df_selected_sorted.groupby('properties.Name').resample('15T').mean()
df_selected_sorted.reset_index(inplace=True)
df_selected_sorted.set_index('properties.PubDate', inplace=True)

In [20]:
test = pd.DataFrame(Parking_coordinates["geometry.coordinates"].to_list(), columns=['lng', 'lat'])
Parking_coordinates = Parking_coordinates.reset_index()

Parking_coordinates = Parking_coordinates.join(test)
Parking_coordinates.drop_duplicates(subset= ['properties.Name'],inplace=True)
Parking_coordinates.set_index('properties.Name', inplace=True)
Parking_coordinates.index.rename('Parking Location', inplace=True)
Parking_coordinates = Parking_coordinates[['lat','lng']]
Parking_coordinates

,lat,lng
Parking Location,,
CE-P05 Euro Parking,52.369880,4.876630
ZO-P03 Mikado,52.310860,4.939850
CE-P02 P+R Olympisch Stadion,52.344400,4.853600
CE-P26 The Bank,52.366058,4.898584
IJDok,52.384820,4.894750
ZD-P02 VUmc (ACTA),52.336200,4.860900
ZU-FP01 Mahlerplein,52.337235,4.873558
CE-P04 Amsterdam Centraal,52.379070,4.897220
CE-P13 Artis,52.368100,4.913680


In [21]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def add_nearby_parkings(radius, center_point_dict, add_to_list):
    """
    radius in km
    center_point_dict needs to be a dictionary with the lon and lat from a location
    add_to_list specify to which list this needs to be added (for example vondelpark)
    """
    latpark = center_point_dict[0]['lat']
    lonpark = center_point_dict[0]['lng']

    # check for every station if it is within 1 km distance of the park
    for parking in range(len(Parking_coordinates)):
        name_parking = Parking_coordinates.iloc[parking].name
        latparking = Parking_coordinates.iloc[parking].lat
        lonparking = Parking_coordinates.iloc[parking].lng

        a = haversine(lonpark, latpark, lonparking, latparking)
        
        if a <= radius:
            add_to_list.append(name_parking)

In [22]:
# Center points of all the parks (https://www.latlong.net/)
all_parks = ['vondelpark_west','vondelpark_oost_3','vondelpark_oost_2',
             'vondelpark_oost_1' 'oosterpark', 'sarphatipark',
            'westerpark_west','westerpark_oost','westerpark_centrum', 
             'westergasfabriek','rembrandtpark_noord', 'rembrandtpark_zuid', 
             'erasmuspark']

vondelpark_west = [{'lat': 52.356496, 'lng': 4.861447}]
vondelpark_oost_3 = [{'lng': 4.869217, 'lat': 52.358252}]
vondelpark_oost_2 = [{'lng': 4.874692, 'lat': 52.359798}]
vondelpark_oost_1 = [{'lng': 4.879652, 'lat': 52.360991}]
oosterpark = [{'lng': 4.920558, 'lat': 52.360098}]
sarphatipark = [{'lng': 4.896375, 'lat': 52.354364}]
westerpark_west = [{'lng': 4.867128, 'lat': 52.387099}]
westerpark_centrum = [{'lng': 4.873268, 'lat': 52.387374}]
westerpark_oost = [{'lng': 4.878379, 'lat': 52.386379}]
westergasfabriek = [{'lng': 4.869769, 'lat': 52.385920}]
rembrandtpark_noord = [{'lng': 4.846573, 'lat': 52.366664}]
rembrandtpark_zuid = [{'lng': 4.846932, 'lat': 52.361161}]
erasmuspark = [{'lng': 4.851909, 'lat': 52.374808}]
amstelpark = [{'lng': 4.894404, 'lat': 52.330409}]
Park_Frankendael = [{'lng': 4.929839, 'lat': 52.350703}]
Beatrixpark = [{'lng': 4.881352, 'lat': 52.342471}]
Flevopark = [{'lng': 4.947881, 'lat': 52.360087}]
Gaasperpark = [{'lng': 4.992192, 'lat': 52.310420}]
Nelson_Mandelapark = [{'lng': 4.963691, 'lat': 52.312204}]
Noorderpark = [{'lng': 4.919606, 'lat': 52.392651}]
Sloterpark = [{'lng': 4.811894, 'lat': 52.366219}]
WH_Vliegenbos = [{'lng': 4.931495, 'lat': 52.388802}]

# Create empty list that needs to be filled later
vondelpark_west_parkings = []
vondelpark_oost_3_parkings = []
vondelpark_oost_2_parkings = []
vondelpark_oost_1_parkings = []
oosterpark_parkings = []
sarphatipark_parkings = []
westerpark_west_parkings = []
westerpark_centrum_parkings = []
westerpark_oost_parkings = []
westergasfabriek_parkings = []
rembrandtpark_noord_parkings = []
rembrandtpark_zuid_parkings = []
erasmuspark_parkings = []
amstelpark_parkings = []
Park_Frankendael_parkings= []
Beatrixpark_parkings = []
Flevopark_parkings = []
Gaasperpark_parkings = []
Nelson_Mandelapark_parkings = []
Noorderpark_parkings = []
Sloterpark_parkings = []
WH_Vliegenbos_parkings = []


# calculate all parkings within 1 km from the park
add_nearby_parkings(1.5,vondelpark_west, vondelpark_west_parkings)
add_nearby_parkings(1,vondelpark_oost_3, vondelpark_oost_3_parkings)
add_nearby_parkings(1,vondelpark_oost_2, vondelpark_oost_2_parkings)
add_nearby_parkings(1,vondelpark_oost_1, vondelpark_oost_1_parkings)
add_nearby_parkings(1,oosterpark,oosterpark_parkings)
add_nearby_parkings(1,sarphatipark,sarphatipark_parkings)
add_nearby_parkings(1,westergasfabriek, westergasfabriek_parkings)
add_nearby_parkings(1,westerpark_west, westerpark_west_parkings)
add_nearby_parkings(1,westerpark_centrum, westerpark_centrum_parkings)
add_nearby_parkings(1,westerpark_oost, westerpark_oost_parkings)
add_nearby_parkings(1,rembrandtpark_noord,rembrandtpark_noord_parkings)
add_nearby_parkings(1,rembrandtpark_zuid,rembrandtpark_zuid_parkings)
add_nearby_parkings(1,erasmuspark,erasmuspark_parkings)
add_nearby_parkings(1,amstelpark,amstelpark_parkings)
add_nearby_parkings(1,Park_Frankendael,Park_Frankendael_parkings)
add_nearby_parkings(1,Beatrixpark,Beatrixpark_parkings)
add_nearby_parkings(1,Flevopark,Flevopark_parkings)
add_nearby_parkings(1,Gaasperpark,Gaasperpark_parkings)
add_nearby_parkings(1,Nelson_Mandelapark,Nelson_Mandelapark_parkings)
add_nearby_parkings(1,Noorderpark,Noorderpark_parkings)
add_nearby_parkings(1,Sloterpark,Sloterpark_parkings)
add_nearby_parkings(1,WH_Vliegenbos,WH_Vliegenbos_parkings)


In [23]:
sarphatipark_parkings

['CE-P33 Albert Cuyp',
 'CE-P30 Heinekenplein',
 'DP-FP05 Ceintuurbaan',
 'DP-FP06 Ferdinand Bolstraat',
 'CE-P31 Prins&Keizer']

In [25]:
df_selected_sorted.to_csv('Parking_CSV')